In [1]:
!pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")



c:\Users\Ty\mlops\mlops_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt")


In [4]:

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()


print("Predicted answer:", model.config.id2label[idx])

# TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [5]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [6]:
from typing import Union
from fastapi import FastAPI, UploadFile
import io
from PIL import Image

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.post("/ask")
def ask(text: str, image: UploadFile):
    content = image.file.read()

    image = Image.open(io.BytesIO(content))
    image = Image.open(image.file)

    result = model_pipeline(text, image)
    return {"answer": result}






In [8]:
import nest_asyncio
nest_asyncio.apply()
from pyngrok import ngrok
import uvicorn

In [ ]:



ngrok.set_auth_token("token")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
config = uvicorn.Config(app, port=8000, log_level="info")
server = uvicorn.Server(config)

import asyncio
# Use asyncio to serve, compatible with nest_asyncio
await server.serve()



Public URL: https://semielliptical-dorothea-uniconoclastically.ngrok-free.dev


INFO:     Started server process [151356]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2600:4040:732c:b00:3039:81e1:ff12:dc06:0 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64267 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:64267 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [151356]
